In [1]:
import os

In [6]:
module_names = ''

class FormGenerator:

  def __init__(self, namespace, route_singular, object_attributes):
    self.namespace = namespace.split('/')[-1]
    self.route_singular = route_singular

    self.object_attributes = object_attributes # Passed as a Python list. e.g.: ['attr_1', 'attr_2']
    self.object_attributes.append('id')
    self.object_attributes = sorted(self.object_attributes)

    self.class_name = self.get_class_name()
    self.attribute_dict = self.get_attribute_dict()
    self.object_attributes = self.get_object_attributes()

    self.lines = []

  def create_form(self):
    for key, value in self.__dict__.items():
     globals()[key] = value

    self.formatted_lines()
    self.write_to_file()

  def get_attribute_dict(self):
    object_attributes = self.object_attributes
    attribute_dict = "{ "

    for attribute in object_attributes:
      attribute_dict += f"{attribute}: {attribute}, "

    attribute_dict = attribute_dict[:-2]

    return attribute_dict + " }"

  def get_object_attributes(self):
    object_attributes = self.object_attributes
    attributes = ""

    for attribute in object_attributes:
      attributes += f":{attribute}, "

    return attributes[:-2]

  def get_class_name(self):
    class_name = ''.join(v.capitalize() for v in self.route_singular.split('_'))
    return f"{self.namespace.capitalize()}::{class_name}"
  
  def formatted_lines(self):
    self.lines = []
    
    with open(f"templates/forms/sample_form.rb", "r") as file:
      lines = file.readlines()

    for line in lines:
      self.lines.append(line.format(**globals()))

  def write_to_file(self):
    directory = f"app/forms/{self.namespace}"

    if not os.path.exists(directory):
      os.makedirs(directory)

    file_path = f"{directory}/{self.route_singular}_form.rb"
    
    with open(file_path, 'w') as file:
      for line in self.lines:
        file.write(line)